In [ ]:
import numpy as np
import pandas as pd 
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from time import time 
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from math import sqrt
from sklearn.metrics import explained_variance_score
from sklearn.metrics import mean_squared_error
import warnings
import torch.optim as optim
from sklearn.model_selection import KFold
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import explained_variance_score
warnings.filterwarnings('ignore')

In [ ]:
train = pd.read_csv('/kaggle/input/tabular-playground-series-aug-2021/train.csv')
test = pd.read_csv('/kaggle/input/tabular-playground-series-aug-2021/test.csv')

In [ ]:
train.head(3)

In [ ]:
test.head(3)

In [ ]:
print('training set details:')
print('-' * 21)
print('shape:')
print()
print(train.shape)
print('*' * 50)
print('null columns:')
print()
print(train.isnull().sum())
print('*' * 50)
print('total nulls:')
print()
print(sum(train.isnull().sum()))
print('*' * 50)
print('info:')
print()
print(train.info())

In [ ]:
print('testing set details:')
print('-' * 20)
print('shape:')
print()
print(test.shape)
print('*' * 50)
print('null columns:')
print()
print(test.isnull().sum())
print('*' * 50)
print('total nulls:')
print()
print(sum(test.isnull().sum()))
print('*' * 50)
print('info:')
print()
print(test.info())

In [ ]:
train.describe()

In [ ]:
train['loss'].describe().to_frame()

In [ ]:
test.describe()

In [ ]:
plt.subplots(figsize = (12,8))
sns.countplot(train['loss'])

In [ ]:
from sklearn.feature_selection import VarianceThreshold as vt
features = train.iloc[:,1:101]
target = train['loss']
v = vt(threshold = 0.8)
v = v.fit(features, target)

cols = v.get_support(indices = True)
cols.shape

In [ ]:
x_train = train.iloc[:, 1:101]
x_test = test.drop('id', axis = 1)

In [ ]:
mm = MinMaxScaler().fit(x_train)
x_train_mm = mm.transform(x_train)
x_test_mm = mm.transform(x_test)

In [ ]:
xtrain_data = pd.DataFrame(x_train_mm)
xtest_data = pd.DataFrame(x_test_mm)
xtrain_data['loss'] = train['loss']

In [ ]:
print('xtrain_data shape:')
print()
print(xtrain_data.shape)
print('*' * 50)
print('xtest_data shape:')
print()
print(xtest_data.shape)

In [ ]:
X_train1 = x_train.copy()
X_test1 = x_test.copy()
num_cols = x_train.columns
for i in num_cols:
    scale = StandardScaler().fit(X_train1[[i]])
    X_train1[i] = scale.transform(X_train1[[i]])
    X_test1[i] = scale.transform(X_test1[[i]])

X_train1["loss"] = train["loss"]

In [ ]:
if "Set" not in X_train1.columns:
    X_train1["Set"] = np.random.choice(["train", "valid"], p =[.8, .2], size=(X_train1.shape[0],))

train_indices = X_train1[X_train1.Set=="train"].index
valid_indices = X_train1[X_train1.Set=="valid"].index

In [ ]:
unused_feature = ['Set']
target = 'loss'
feature = [ col for col in X_train1.columns if col not in unused_feature+[target]] 

In [ ]:
#train_data
X_train = X_train1[feature].values[train_indices]
y_train = X_train1[target].values[train_indices].reshape(-1, 1)
#valid data 
X_valid = X_train1[feature].values[valid_indices]
y_valid = X_train1[target].values[valid_indices].reshape(-1, 1)

In [ ]:
sample = pd.read_csv('/kaggle/input/tabular-playground-series-aug-2021/sample_submission.csv')

In [ ]:
paramsXGB = {'max_depth': 10, 'learning_rate': 0.010512283852839102, 'n_estimators': 2432, 'min_child_weight': 185, 'gamma': 0.00010339779073732135, 'alpha': 0.00573215966018785, 'lambda': 0.00013592165632140884, 'colsample_bytree': 0.5825502178882395, 'subsample': 0.6245584427453496,
              'tree_method': 'gpu_hist',
              'booster': 'gbtree',
              'random_state': 228,
              'use_label_encoder': False,
              'eval_metric': 'rmse'}

In [ ]:
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.model_selection import StratifiedKFold, KFold, LeaveOneGroupOut

In [ ]:
xgb_model = XGBRegressor(**paramsXGB)

In [ ]:
X = train.drop('loss', axis = 1)
y = train['loss']

sc = StandardScaler()
X[X.columns.tolist()] = sc.fit_transform(X[X.columns.tolist()])
test[test.columns.tolist()] = sc.fit_transform(test[test.columns.tolist()])

X.head(3)

In [ ]:
xgb_model.fit(X_train, y_train)
    
predictions = xgb_model.predict(X_test1)

In [ ]:
predictions = pd.DataFrame(predictions) # from array to dataframe
sample['loss'] = predictions
sample.to_csv('xgb_submission.csv',index=False)
sample